In [2]:
import sqlite3 as sq
from collections import defaultdict
from nltk import ngrams
import tqdm
import glob, os
import shelve
import random


In [2]:
sentence='hi i am kapil'.split()
for i,j,k in ngrams(sentence,n=3 ,pad_right=True, pad_left=True):
    print(i,j,k)

None None hi
None hi i
hi i am
i am kapil
am kapil None
kapil None None


# The method
Here we describe the method for trigram model. Similar method is used for 4gram model
- The frequencies of bigram and trigram are stored in a database. We didnt use dictionary since it had to be loaded into memory and it was slowing down the pc. Instead, by using database we are storing the data onto disk rather than memory.
- Training on 2000 documents seemed to be sufficient. Also training on whole data led to very large database size.
- Then we compute the conditional probabilities.
- Finally we generate sentences..

#### Intitializing the databases

In [6]:
fourgram=sq.connect('fourgram_model')
trigram=sq.connect('trigram_model')
bigram=sq.connect('bigrams')
f=fourgram.cursor()
t=trigram.cursor()
b=bigram.cursor()

In [19]:
try:
    b.execute('DROP TABLE bigrams')
except:
    b.execute('''CREATE TABLE bigrams
             (w1 text, w2 text, num FLOAT DEFAULT 0,PRIMARY KEY(w1,w2))''')
try:
    t.execute('DROP TABLE trigrams')
except:
    t.execute('''CREATE TABLE trigrams
             (w1 text, w2 text, w3 text, num FLOAT DEFAULT 0, prob FLOAT,PRIMARY KEY(w1,w2,w3))''')



In [4]:
try:
    f.execute('DROP TABLE fourgrams')
except:
    f.execute('''CREATE TABLE fourgrams
             (w1 text, w2 text, w3 text, w4 text, num FLOAT DEFAULT 0, prob FLOAT,PRIMARY KEY(w1,w2,w3,w4))''')

In [9]:
data='../assignment 1/processed_text'

#### Here we store all the frequencies of trigrams and bigrams into the databases

In [20]:

i=0
for fname in tqdm.tqdm(os.listdir(f'{data}//')):
    i+=1
    if i==2000:
        break
    with open(f'{data}//{fname}') as f:
        text=f.read()
    sentences=text.split('.')
    for s in sentences:
        words=s.split()
        if len(words)<3:
            continue
        for w1,w2,w3 in ngrams(words,n=3,pad_right=True, pad_left=True,left_pad_symbol='\x00', right_pad_symbol='\x00'):
            if w1==w2==w3:
                continue
            t.execute('INSERT OR IGNORE INTO trigrams (w1, w2,w3) VALUES (?,?,?)',(w1,w2,w3))
            t.execute('UPDATE trigrams SET num = num+1 WHERE (w1,w2,w3) IS (?,?,?)',(w1,w2,w3))
            b.execute('INSERT OR IGNORE INTO bigrams (w1, w2) VALUES (?,?)',(w1,w2))
            b.execute('UPDATE bigrams SET num = num+1 WHERE (w1,w2) IS (?,?)',(w1,w2))


  1%|▉                                                                         | 1999/153862 [05:37<7:07:41,  5.92it/s]


#### Updating the probabilities

In [21]:
trigram.commit()
bigram.commit()
t2=trigram.cursor()
b.execute('SELECT * FROM bigrams') 
for row in tqdm.tqdm(b):
    t2.execute('UPDATE trigrams SET prob=num/? where (w1,w2) IS (?,?)',(row[2],row[0],row[1]))
trigram.commit()

1711842it [01:02, 27375.70it/s]


#### Lets generate sentences!

In [37]:
t=trigram.cursor()
ans=''

for i in range(5):
    curr=['\x00','\x00']
    ans=''
    while True:
        res=list(t.execute('SELECT prob,w3 FROM trigrams WHERE (w1,w2) IS (?,?) ',curr))
        res.sort()
        if i==0:
            last=res[-1][1]
        else:
            last=res[-min(random.randint(1,4),len(res))][1]
        if last=='\x00':
            break
        ans+=last+' '
        curr.pop(0)
        curr.append(last)
    if i==0:
        print("Sentence generated with most occuring starting word is ---",ans,'---')
        print('')
    else:
        if i==1:
            print("Some random sentences generated using one among first 3 predictions are")
        print('')
        print(ans)

Sentence generated with most occuring starting word is --- the copyright holder for this preprint this version posted may  ---

Some random sentences generated using one among first 3 predictions are

this may be a promising therapeutic tools are required to determine which of them were located at residues and which were used for all of the covid outbreak 

we found the mean time between onset and hospital admission 

we have a financial incentive for practices risp and a significant increase p and higher than in those countries that are more likely to report the average of all the above two guidelines 

in this paper we introduce a new approach provided private multisite fmri analysis through numerical results for sarscov and sarscov spike protein s and s the summation of binding of viral infection 


#### Storing the frequencies of four gram model

In [10]:
i=0
for fname in tqdm.tqdm(os.listdir(f'{data}//')):
    i+=1
    if i==2000:
        break
    with open(f'{data}//{fname}') as fi:
        text=fi.read()
    sentences=text.split('.')
    for s in sentences:
        words=s.split()
        if len(words)<3:
            continue
        for w1,w2,w3,w4 in ngrams(words,n=4,pad_right=True, pad_left=True,left_pad_symbol='\x00', right_pad_symbol='\x00'):
            if (w1==w2 or w2==w3 or w3==w4) and (w1!='\x00' and w2!='\x00' and w3!='\x00' and w4!='\x00'):
                continue
            f.execute('INSERT OR IGNORE INTO fourgrams (w1, w2,w3,w4) VALUES (?,?,?,?)',(w1,w2,w3,w4))
            f.execute('UPDATE fourgrams SET num = num+1 WHERE (w1,w2,w3,w4) IS (?,?,?,?)',(w1,w2,w3,w4))

  1%|▉                                                                         | 1999/153862 [04:02<5:07:33,  8.23it/s]


#### Converting into probabilities

In [27]:
fourgram.commit()
f2=fourgram.cursor()
t.execute('SELECT * FROM trigrams') 
for row in tqdm.tqdm(t):
    if row[0]==row[1]==row[2]=='\x00':
        f2.execute('UPDATE fourgrams SET prob=num/? where (w1,w2,w3) IS (?,?,?)',(1,row[0],row[1],row[2]))
    else:
        f2.execute('UPDATE fourgrams SET prob=num/? where (w1,w2,w3) IS (?,?,?)',(row[-2],row[0],row[1],row[2]))
f2.execute('UPDATE fourgrams SET prob=num/? where (w1,w2,w3) IS (?,?,?)',(1,'\x00','\x00','\x00'))
fourgram.commit()

4366468it [01:54, 38130.64it/s]


#### Lets Generate sentences!

In [33]:
f=fourgram.cursor()
ans=''
for i in range(5):
    curr=['\x00','\x00','\x00']
    ans=''
    while True:
        res=list(f.execute('SELECT prob,w4 FROM fourgrams WHERE (w1,w2,w3) IS (?,?,?) ',curr))
        res.sort()
        if i==0:
            last=res[-1][1]
        else:
            last=res[-min(random.randint(1,4),len(res))][1]
        if last=='\x00':
            break
        ans+=last+' '
        curr.pop(0)
        curr.append(last)
    if i==0:
        print("Sentence generated with most occuring starting word is ---",ans,'---')
        print('')
    else:
        if i==1:
            print("Some random sentences generated using one among first 3 predictions are")
        print('')
        print(ans)

Sentence generated with most occuring starting word is --- the copyright holder for this preprint this version posted may  ---

Some random sentences generated using one among first 3 predictions are

in the first year of life for many american families board of governors 

the copyright for prismap including checklist is held by one individual is now reduced by the proportion of publichealth expenditure and fiscal expenditure is shown in figure the number of cases 

we also found evidence for recombination deduced from the sequence read archive sra database accession no 

the first step is that each view is now a critical function and that the k antagonist mr indicating the drug acted like a u agonist in this situation with the focus of this paper 


# Observations
- The senetences generated using the highest probability words were same for both trigram and 4gram model
- Looking at the senetences generated by using random element from top 3 highest probability words, it seems 4gram model performs better. This might be bcoz 4gram performs 'lookahead' better than trigrams. Also the crucial thing is that the dataset for 4gram was large enough. Otherwise, it would simple have overfit.

## Answers to other questions

##### Efficient ways to handle the large set of parameters
- We used database instead of dictionary for this purpose. Hence the data was loaded onto disk rather than memory and could be efficiently retrieved when generating sentences.

##### Is it possible to run any portion of the code in parallel? If yes, which part of the code? If no, why?
- We can use several threads to process multiple files to generate the frequencies of ngrams in parallel